In [1]:
import pandas as pd


In [2]:
# Eligible est las liste fournie par les RH donnat les personnes éligibles au gift voucher
eligibles=pd.read_excel("employee-vouchers.xlsx", dtype={'Emp No': "str"})
eligibles["mail"]=eligibles["Emp Mail Id"]+"@infosys.com"
eligibles["mail"]=eligibles["mail"].str.lower()
# Construction de la colonne nom simple qui contient une concaténation du prénom et du nom en minuscule
eligibles["nomsimple"]=eligibles["Empname"].str.lower().str.replace("é","e").replace("è","e").replace("ì","i")
eligibles

,Emp No,Empname,Emp Mail Id,Commentaire,mail,nomsimple
0,4484,Raghu R. Hemmige,RAGHURHEMMIGE,NaN,raghurhemmige@infosys.com,raghu r. hemmige
1,15536,Antony Menacherry,Antony_Menacherry,NaN,antony_menacherry@infosys.com,antony menacherry
2,15958,Sachidananda Mohanty,Sachidananda_Mohanty,NaN,sachidananda_mohanty@infosys.com,sachidananda mohanty
3,17521,Ajit Venkatesh Kamath,Ajit_Kamath,NaN,ajit_kamath@infosys.com,ajit venkatesh kamath
4,19406,Shenbagarajan Dharmar,Shenbagarajan_d,NaN,shenbagarajan_d@infosys.com,shenbagarajan dharmar
...,...,...,...,...,...,...
437,1338805,Paul Gervais,paul.gervais,Ancien stagiaire - plus de 7 mois en 2023 en tout,paul.gervais@infosys.com,paul gervais
438,1338806,Roméo Malivert,romeo.malivert,Ancien stagiare - plus de 7 mois en 2023 en tout,romeo.malivert@infosys.com,romeo malivert
439,1338808,Céline Caillierez,celine.caillierez,Non éligible concernant expérience Infosys. A ...,celine.caillierez@infosys.com,celine caillierez
440,1339890,Arnau Garcia Cases,arnau.cases,Ancien stagiaire - plus de 7 mois en 2023 en tout,arnau.cases@infosys.com,arnau garcia cases


In [3]:
#Users est pris d'un extract Comitéo. : Liste des utilisateurs actifs et désactivés (sans adresse)
users=pd.read_csv("userscomiteo.csv",sep=";", dtype={'Matricule': "str"})

subset = ['Mail', 'Mail 2',"Matricule"]
users.loc[:, subset] = users.loc[:, subset].fillna("")


users["mail"]=""
users=users[users["Etat du salarié"]=="Actif"]

#Construction de la colonne mail qui sera utilisée pour le rapprochement. Elle contiendra le mail qui contient infosys.com
for i, row in users.iterrows():
    if "@infosys.com" in row["Mail"]:
        users.loc[i,"mail"]=row["Mail"]
    elif "@infosys.com" in  row["Mail 2"]:
        users.loc[i,"mail"]=row["Mail 2"]
# Minuscule
users["mail"]=users["mail"].str.lower()
# Construction de la colonne nom simple qui contient une concaténation du prénom et du nom en minuscule
users["nomsimple"]=users["Prénom"]+" "+users["Nom"]
users["nomsimple"]=users["nomsimple"].str.lower()

users
users[users.mail.str.contains("rec")]["Identifiant"]

134         jmaraval4
677       mbenkaddour
785    virginie_leroy
Name: Identifiant, dtype: object

In [4]:

from IPython.core.display import HTML
trouves=[]
nontrouves=[]

#Premier rapprochement sur l'email
for i,ligne in eligibles.iterrows():   
    if  len(users.loc[users["mail"]==ligne.mail])>0:
        #print ("Trouvé ")
        indextrouve=users.loc[users["mail"]==ligne.mail].Identifiant.index[0]
        #print( indextrouve)
        trouves.append(indextrouve)
    else:
        #print ("Non trouvé")
        nontrouves.append(i)

#Second rapprochement sur le matricule.
nontrouvesrestant=[]
for i,ligne in eligibles.loc[nontrouves].iterrows():   
    #print(ligne["Emp No"])
    if  len(users.loc[users["Matricule"]==ligne["Emp No"]])>0:
        #print ("Trouvé ")
        indextrouve=users.loc[users["Matricule"]==ligne["Emp No"]].index[0]
        #print( indextrouve)
        trouves.append(indextrouve)
    else:
        #print ("Non trouvé")
        nontrouvesrestant.append(i)

nontrouves=[]
for i,ligne in eligibles.loc[nontrouvesrestant].iterrows():   
    #print(ligne["Emp No"])
    if  len(users.loc[users["nomsimple"]==ligne["nomsimple"]])>0:
        print ("Trouvé ")
        indextrouve=users.loc[users["nomsimple"]==ligne["nomsimple"]].index[0]
        #print( indextrouve)
        trouves.append(indextrouve)
    else:
        #print ("Non trouvé")
        nontrouves.append(i)

    
eligibles.loc[nontrouves]  

users.loc[trouves,["Matricule","Identifiant"]].to_excel("users-chequecadeau.xlsx")

display(HTML("<h2>Employés non trouvés dans comitéo</h2>"))

eligibles.loc[nontrouves]  

Trouvé 
Trouvé 


,Emp No,Empname,Emp Mail Id,Commentaire,mail,nomsimple


In [5]:
#trouves not in users.index


indexusers = users.index.values.tolist()

usersersindexexclus = []
for element in indexusers:
    if element not in trouves:
        usersersindexexclus.append(element)
 
usersexclus=users.loc[usersersindexexclus]
usersexclus[~usersexclus["Statut salarié"].isna()]


usersexclus.to_excel("excluded-employees.xlsx")

In [20]:
import pandas as pd
from datetime import datetime
def  parsedate(x):
    if len(x)>10:
#        print(x)
        return datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
    else:
        try:
            return datetime.strptime(x, '%Y-%m-%d')
        except:
            return datetime.strptime(x, '%d-%m-%Y')
        
        
#dateparse = lambda x: datetime.strptime(x, '%Y-%m-%d')

ayantsdroit=pd.read_csv("20231218-ayantdroit.csv",sep=";", dtype={'Matricule': "str"},parse_dates=["Date de naissance ayant droit","Date de création"], date_parser=parsedate)
#ayantsdroit=pd.read_csv("ayantdroit.csv",sep=";", dtype={'Matricule': "str"},parse_dates=["Date de naissance ayant droit"], date_format="%Y-%m-%d")
ayantsdroit=ayantsdroit[ayantsdroit["Etat du salarié"]=="Actif"]
ayantsdroit=ayantsdroit[ayantsdroit["Type"]=="enfant"]
ayantsdroit=ayantsdroit[ayantsdroit["Date de création"] > datetime.strptime('2023-12-04 09:45','%Y-%m-%d %H:%M')]


ayantsdroit=ayantsdroit[ayantsdroit["Date de naissance ayant droit"]>datetime.strptime('2007-12-15','%Y-%m-%d')]
ayantsdroit=ayantsdroit[ayantsdroit["Date de naissance ayant droit"]<datetime.strptime('2022-12-15','%Y-%m-%d')]
ayantsdroit[ayantsdroit.Nom=="Tanguy"]
ayantsdroit.to_excel("20231204-Beneficiaires-enfant-cadeau.xlsx")
ayantsdroit.to_csv("20231204-Beneficiaires-enfant-cadeau.csv")
ayantsdroit


/tmp/ipykernel_687/1784478697.py:16: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ayantsdroit=pd.read_csv("20231218-ayantdroit.csv",sep=";", dtype={'Matricule': "str"},parse_dates=["Date de naissance ayant droit","Date de création"], date_parser=parsedate)


,Matricule,Identifiant,Civilité,Prénom,Nom,Etat du salarié,Mail,Site,Type de contrat,Téléphone 1,...,Prénom ayant droit,Nom ayant droit,Date de création,Genre,Type,Statut salarié et ayant droit,Date de naissance ayant droit,Rattachement fiscal,Rattachement familiale,Etat
561,719063,rnarayan_madathil_cochiyil,M,Rahul Narayan,Madathil Cochiyil,Actif,raulkrish@gmail.com,Lyon,NaN,NaN,...,Tanvi,Rahul,2023-12-04 22:05:58,Fille,enfant,"Regul_Cadolib_Noel_2020,NA_2021_100,Cadolib_No...",2016-09-26,Oui,Oui,Non Validé
564,1001874,bijayant.jha,M,Bijayant,Kumar Jha,Actif,bijayant.jha@infosys.com,Toulouse,NaN,NaN,...,Adhya,Vats,2023-12-07 19:50:07,Fille,enfant,NA_70_Dec23,2018-12-04,Non,Oui,Non Validé
567,1313883,olivier.marvillet,NaN,Olivier,Marvillet,Actif,olivier.marvillet@infosys.com,Paris,NaN,NaN,...,FEDOR,MARVILLET,2023-12-09 13:16:17,Garçon,enfant,NA_70_Dec23,2010-10-31,Oui,Oui,Non Validé
568,1313883,olivier.marvillet,NaN,Olivier,Marvillet,Actif,olivier.marvillet@infosys.com,Paris,NaN,NaN,...,Arsène,Marvillet,2023-12-09 13:16:56,Garçon,enfant,NA_70_Dec23,2012-11-15,Oui,Oui,Non Validé
570,1045733,thibaut.fluteaux,M,Thibaut,Fluteaux,Actif,thibaut.fluteaux@infosys.com,Toulouse,NaN,NaN,...,Léa,Fluteaux,2023-12-12 15:56:21,Garçon,enfant,"Cadolib_Noel_170,NA_2021_100,Cadolib_Noel_Adul...",2022-01-02,Oui,Oui,Non Validé
571,705717,nkathamuthu_palaniswamy,M,Nagarajan,Kathamuthu Palaniswamy,Actif,nagarajan.p03@infosys.com,Paris,NaN,NaN,...,Athulya Namrita,NAGARAJAN KAVITHA,2023-12-16 22:04:30,Fille,enfant,"Cadolib_Noel_170,Cadolib_NE_2021_30,NA_2021_10...",2022-05-28,Oui,Oui,Non Validé


In [7]:
#764202
eligibles[eligibles["Emp No"]=="764202"]

,Emp No,Empname,Emp Mail Id,Commentaire,mail,nomsimple
